In [2]:
import pandas as pd
import numpy as np

In [3]:
## Read raw file

folder_path = '/Users/altay.amanbay/Desktop/new node booster/experiments/3a.1 - Nets train/5 train model - keras (unigrams)/1_data/'
file = 'sampled_descriptionary_sample_size_5000.csv'

df = pd.read_csv(folder_path + file)
#df.rename(columns={'\ufeff"description_mod1"': 'items'}, inplace=True)
df.head(2)

,index,description,category_id,category_path
0,0,!iT Jeans Maternity Skinny Jeans Dark Wash M,100,Apparel & Accessories > Apparel > Maternity
1,1,1822 Denim 'Butter' Maternity Skinny Jeans Rin...,100,Apparel & Accessories > Apparel > Maternity


In [4]:
## FUNCTIONS

def get_token_combinations_n_stats_all(certain_cat_descriptions):
    
    token_combinations_list = []
    dscr_tokens_stats = defaultdict(lambda:0)
    for dscr in certain_cat_descriptions.description:   # item descriptions
        dscr_lower_case = dscr.lower()
        dscr_tokens_list = "".join((char if char.isalnum() else " ") for char in dscr_lower_case).split()

        for L in range(1, len(dscr_tokens_list)+1):
            for token_combination in itertools.combinations(dscr_tokens_list, L):
                token_combinations_list.append(token_combination)
                
    for token_combination in token_combinations_list:
        token_combination = list(token_combination)
        token_combination_set = frozenset(token_combination)
        dscr_tokens_stats[token_combination_set] += 1

    return dscr_tokens_stats

def get_token_combinations_n_stats_3_4(certain_cat_descriptions, items_column):
    
    token_combinations_list = []
    dscr_tokens_stats = defaultdict(lambda:0)
    for dscr in certain_cat_descriptions[items_column]:   # item descriptions
        dscr_lower_case = dscr.lower()
        dscr_tokens_list = "".join((char if char.isalnum() else " ") for char in dscr_lower_case).split()

        if(len(dscr_tokens_list) >= 4):
            for token_combination in itertools.combinations(dscr_tokens_list, 3):
                token_combinations_list.append(token_combination)
            for token_combination in itertools.combinations(dscr_tokens_list, 4):
                token_combinations_list.append(token_combination)

                
    for token_combination in token_combinations_list:
        token_combination = list(token_combination)
        token_combination_set = frozenset(token_combination)
        dscr_tokens_stats[token_combination_set] += 1

    return dscr_tokens_stats


def apriori_all(certain_cat_descriptions, minsupp = 0.05):
    dscr_tokens_stats = get_token_combinations_n_stats_all(certain_cat_descriptions)
    
    keys = dscr_tokens_stats.keys()
    items_count = certain_cat_descriptions.shape[0]
    dscr_tokens_stats_cut = {key:(dscr_tokens_stats[key], dscr_tokens_stats[key]/items_count) for key in keys if dscr_tokens_stats[key]/items_count >= minsupp}

    return dscr_tokens_stats_cut

def apriori_3_4(certain_cat_descriptions, items_column, minsupp = 0.05, maxsupp = 1.0):
    dscr_tokens_stats = get_token_combinations_n_stats_3_4(certain_cat_descriptions, items_column)
    
    keys = dscr_tokens_stats.keys()
    items_count = certain_cat_descriptions.shape[0]
    dscr_tokens_stats_cut = {key:(dscr_tokens_stats[key], dscr_tokens_stats[key]/items_count) \
                             for key in keys if dscr_tokens_stats[key]/items_count >= minsupp and dscr_tokens_stats[key]/items_count <= maxsupp}

    return dscr_tokens_stats_cut



# Marks truncated items as 'yes' in new 'truncated' column
def mark_truncated_items(data_df, items_column):
    # Add column 'truncated'
    truncated_marked = ['no'] * data_df.shape[0]
    
    # Mark truncated items as 'yes' in list
    for i in range(data_df.shape[0]):
        start = time.time()
        for j in range(data_df.shape[0]):
            if(data_df.ix[i,items_column] in data_df.ix[j,items_column] and data_df.ix[i,items_column] != data_df.ix[j,items_column]):
                truncated_marked[i] = 'yes'
                break
        if(i % 10 == 0):
            print(i)
            print('Took %g sec: ' % (time.time() - start))
    
    return truncated_marked

# Returns token frequencies as dataframe
def get_token_frequencies(data_df, items_column):
    # Get token frequencies in Series
    cnt_sr = data_df[items_column].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)

    # Name frequencies column as 'freq'
    cnt_df = pd.DataFrame(cnt_sr, columns = ['freq'])

    # Sort frequencies
    cnt_df.sort_values(['freq'], ascending=[False], inplace=True)

    # Convert tokens index into column, create new numeric index
    cnt_df.reset_index(level=0, inplace=True)

    # Rename tokens column as 'items'
    cnt_df.rename(columns={'index': 'tokens'}, inplace=True)
    
    # Cast float to int
    cnt_df['freq'] = cnt_df['freq'].astype(np.int64)
    
    return cnt_df

In [5]:
## Preprocess of item descriptions
import re
import time


def process_string(sentence_):
    printable_ = 'abcdefghijklmnopqrstuvwxyz0123456789 '
    sentence_processed = "".join((char if char in printable_ else "") for char in sentence_.lower())
    
    return sentence_processed

df_pos['description_lc_ano'] = df_pos['description'].apply(lambda x: process_string(x))      

#2b after step #2a, need to replace multiple spaces to single space
df_pos['description_lc_ano'] = df_pos['description_lc_ano'].apply(lambda x: re.sub(' +',' ',x))  


df_pos.head()

NameError: name 'df_pos' is not defined

In [90]:
'AA'.lower()


'aa'

In [ ]:
start = time.time()

categories_unique_ar = pd.unique(df.loc[:,['category_path']].values)
categories_unique_ar = [['Apparel & Accessories > Apparel > Maternity']]

df_pos['truncated_lc_ano'] = 'no'
items_column = 'description_lc_ano'

for cat in categories_unique_ar:
    print('Category name:',cat[0])
    start_idx = min(df.loc[(df.category_path==cat[0]),:].index.values.tolist())
    end_idx = max(df.loc[(df.category_path==cat[0]),:].index.values.tolist())
    print('start index:',start_idx)
    print('end index:',end_idx)
    
    # Mark truncated items as 'yes' in list
    for i in range(start_idx,end_idx+1):
        #for j in range(i+1,end_idx+1):
        for j in range(start_idx,end_idx+1):
            if(df_pos.ix[i,items_column] in df_pos.ix[j,items_column] and df_pos.ix[i,items_column] != df_pos.ix[j,items_column]):
            #if(df_pos.ix[i,items_column] in df_pos.ix[j,items_column]):
                df_pos.ix[i,'truncated_lc_ano'] = 'yes'
                break

print('Preprocessing took: %g sec'% (time.time() - start))

In [42]:
d = {'aa':1,'bb':7,'cc':2,'dd':3,'ee':4,'ff':5,'gg':6}

keys = list(d.keys())
np.random.shuffle(keys)
keys
#dict([(key, d[key]) for key in list(keys)])

['dd', 'ee', 'ff', 'aa', 'cc', 'gg', 'bb']

In [87]:
df_pos[df_pos.truncated_lc_ano == 'yes']
#df_pos[df_pos.description_lc_ano=='a pea in the pod jeans maternity secret fit be...']

,index,description,category_id,category_path,description_lc_ano,truncated_lc_ano
24,24,3401 mama j super skinny maternity jeans,100,Apparel & Accessories > Apparel > Maternity,3401 mama j super skinny maternity jeans,yes
39,39,a pea in the pod jeans maternity secret fit be...,100,Apparel & Accessories > Apparel > Maternity,a pea in the pod jeans maternity secret fit be...,yes
96,96,avedon coated maternity jeans,100,Apparel & Accessories > Apparel > Maternity,avedon coated maternity jeans,yes
97,97,avedon maternity jeans,100,Apparel & Accessories > Apparel > Maternity,avedon maternity jeans,yes
292,292,florence maternity skinny jeans,100,Apparel & Accessories > Apparel > Maternity,florence maternity skinny jeans,yes
367,367,james jeans bootcut maternity jean carolina 27,100,Apparel & Accessories > Apparel > Maternity,james jeans bootcut maternity jean carolina 27,yes
481,481,jimmy jimmy maternity boyfriend jean / rebel / 28,100,Apparel & Accessories > Apparel > Maternity,jimmy jimmy maternity boyfriend jean rebel 28,yes
482,482,jimmy jimmy maternity skinny jeans,100,Apparel & Accessories > Apparel > Maternity,jimmy jimmy maternity skinny jeans,yes
486,486,liz lange for target maternity over-the-belly ...,100,Apparel & Accessories > Apparel > Maternity,liz lange for target maternity overthebelly bo...,yes
490,490,liz lange for target maternity over-the-belly ...,100,Apparel & Accessories > Apparel > Maternity,liz lange for target maternity overthebelly bo...,yes


In [19]:
import pandas as pd
import logging
import sys
import os.path
import time
from datetime import datetime
from IPython.display import display, HTML
from gensim.models import word2vec, Phrases
from nltk.corpus import brown, movie_reviews, treebank    
    
    
def print_list(ls):
    for i in ls:
        print(i)
    print()    
    

sentences_ls_1 = [['aa','class_a']*1
                ,['ab','class_a']*1
                ,['bb','class_b']*1
                ,['bc','class_b']*1
                #,['I','love','NY']
                #,['I','love','NY']
               ]
sentences_ls_2 = [['class_a','aa']*100
                ,['class_a','ab']*5000
                ,['class_b','bb']*1
                ,['class_b','bc']*2000
                #,['I','love','NY']
                #,['I','love','NY']
               ]
ext = [['i','like','london']]*10000

#sentences_ls.extend(ext)
#print(sentences_ls.append(ext))
#print(ext)
#sentences_ls.extend(ext)
#print(sentences_ls)
    
#3 Set parameters
num_features = 2     # Word vector dimensionality. (default=100)
min_word_count = 0    # ignore words that are less than count of min_word_count in corpus. (default=5)
num_workers = 2       # Number of threads to run in parallel
context = 1           # Context window size
downsampling = 0.1#1e-3   # Downsample setting for frequent words
iter_ = 100
sg = 1                # if 1 skip-gram technique is used, else CBoW. (default=0)

#4 Start training Word2Vec model
model = word2vec.Word2Vec(sentences_ls_1, sg = 1, workers = num_workers, size = num_features, iter = iter_, min_count = min_word_count, window = context, sample = downsampling, seed = 1)


#6 Test Word2Vec model
test_words = ['I','like','love','Paris','London']
print('='*100)
print('Test for '+str(len(test_words))+' words:')
#for word in test_words:
for word in model.vocab.keys():
    print('   Similar to ' + word)
    print_list(model.most_similar(word, topn=3))

for word in model.vocab.keys():
    print(word,':',model[word])
# if(len(sentences_ls) < 4):
#    print_list(sentences_ls)

Test for 5 words:
   Similar to class_b
('class_a', 0.8538116216659546)
('bb', 0.705333948135376)
('bc', 0.6561290621757507)

   Similar to ab
('aa', 0.9986451268196106)
('class_a', 0.6006267070770264)
('class_b', 0.0966012179851532)

   Similar to bc
('bb', 0.9977417588233948)
('class_b', 0.6561290621757507)
('class_a', 0.167354017496109)

   Similar to bb
('bc', 0.9977418780326843)
('class_b', 0.7053340077400208)
('class_a', 0.23319467902183533)

   Similar to class_a
('class_b', 0.8538116216659546)
('aa', 0.6414181590080261)
('ab', 0.6006267070770264)

   Similar to aa
('ab', 0.9986451268196106)
('class_a', 0.6414182186126709)
('class_b', 0.14826416969299316)

class_b : [-1.51037633  1.13236821]
ab : [-0.78824401 -0.86313665]
bc : [-0.10705756  1.47708988]
bb : [-0.19352511  1.37757969]
class_a : [-1.80196249  0.17314535]
aa : [-0.90754092 -0.89538842]


In [139]:
model.similarity('NY','Paris')

0.39910241124842316

In [23]:
import pandas as pd
import logging
import sys
import os.path
import time
from datetime import datetime
from IPython.display import display, HTML
from gensim.models import word2vec, Phrases
from nltk.corpus import brown, movie_reviews, treebank    
    
    
def print_list(ls):
    for i in ls:
        print(i)
    print()    
    

sentences_ls_1 = [['a1','class_a']*100
                ,['a2','class_a']*5000
                ,['b1','class_b']*1
                ,['b2','class_b']*2000
                ,['c1','class_c']*1
                #,['I','love','NY']
                #,['I','love','NY']
               ]
sentences_ls_2 = [['class_a','a1']*100
                ,['class_a','a2']*5000
                ,['class_b','b1']*1
                ,['class_b','b2']*2000
                ,['class_c','c1']*1
                #,['I','love','NY']
                #,['I','love','NY']
               ]
ext = [['i','like','london']]*10000

#sentences_ls.extend(ext)
#print(sentences_ls.append(ext))
#print(ext)
#sentences_ls.extend(ext)
#print(sentences_ls)
    
#3 Set parameters
num_features = 64     # Word vector dimensionality. (default=100)
min_word_count = 0    # ignore words that are less than count of min_word_count in corpus. (default=5)
num_workers = 2       # Number of threads to run in parallel
context = 1           # Context window size
downsampling = 0.1#1e-3   # Downsample setting for frequent words
iter_ = 10
sg = 1                # if 1 skip-gram technique is used, else CBoW. (default=0)

#4 Start training Word2Vec model
model = word2vec.Word2Vec(sentences_ls_1, sg = 1, workers = num_workers, size = num_features, iter = iter_, min_count = min_word_count, window = context, sample = downsampling, seed = 1)


#6 Test Word2Vec model
test_words = ['I','like','love','Paris','London']
print('='*100)
print('Test for '+str(len(test_words))+' words:')
#for word in test_words:
for word in model.vocab.keys():
    print('   Similar to ' + word)
    print_list(model.most_similar(word, topn=3))

# if(len(sentences_ls) < 4):
#    print_list(sentences_ls)

Test for 5 words:
   Similar to c1
('class_c', 0.9430673718452454)
('a2', 0.6973112225532532)
('class_b', 0.6967525482177734)

   Similar to b2
('b1', 0.9557626843452454)
('class_c', 0.7044070959091187)
('c1', 0.4633413851261139)

   Similar to class_a
('c1', 0.3400857150554657)
('class_c', 0.3100892901420593)
('class_b', 0.1478729546070099)

   Similar to class_b
('class_c', 0.7104061841964722)
('c1', 0.6967525482177734)
('b1', 0.43376681208610535)

   Similar to a2
('c1', 0.6973112225532532)
('class_c', 0.5816522240638733)
('a1', 0.4656508266925812)

   Similar to a1
('c1', 0.6357032060623169)
('class_c', 0.5547725558280945)
('a2', 0.4656508266925812)

   Similar to class_c
('c1', 0.9430673718452454)
('class_b', 0.7104061841964722)
('b2', 0.7044072151184082)

   Similar to b1
('b2', 0.9557627439498901)
('class_c', 0.5975402593612671)
('class_b', 0.43376681208610535)



In [93]:
from keras.preprocessing import sequence

char_set = 'abcdefghijklmnopqrstuvwxyz0123456789'
char_set = 'abc'
vocab_size = len(char_set)
print('total chars:', len(char_set))

char_indices = dict((c, i) for i, c in enumerate(char_set))
indices_char = dict((i, c) for i, c in enumerate(char_set))
#char_indices[['a','a']]

def vectorize_sentences(data, char_indices):
    data_X = []
    for sentence in data:
        sent_X = []
        for word in sentence:
            x = [char_indices[c] for c in word]
            x2 = np.eye(len(char_indices))[x]
            sent_X.append(x2)
        data_X.append(sent_X)
    return data_X#(sequence.pad_sequences(X, maxlen=4))

def vectorize_sentences(data, char_indices):
    X = []
    for sentences in data:
        x = [char_indices[w] for w in sentences]
        x2 = np.eye(len(char_indices))[x]
        X.append(x2)
    return (sequence.pad_sequences(X, maxlen=5))

data = [['a','b'],['c','b']]
vectorize_sentences(data, char_indices)



total chars: 3


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0]]], dtype=int32)

In [86]:
data_df = pd.read_csv("/Users/altay.amanbay/Desktop/sentiment labelled sentences/imdb_labelled.txt", delimiter='\t', header=None, quoting=3, names=['review','label'])
data_df.head()


,review,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [87]:
docs = []
sentences = []
sentiments = []

for sentences, sentiment in zip(data_df.review, data_df.label):
    sentences_cleaned = [sent.lower() for sent in sentences]
    docs.append(sentences_cleaned)
    sentiments.append(sentiment)

In [90]:
def vectorize_sentences(data, char_indices):
    X = []
    for sentences in data:
        x = [char_indices[w] for w in sentences]
        x2 = np.eye(len(char_indices))[x]
        X.append(x2)
    return (pad_sequences(X, maxlen=20))

vectorize_sentences()

['n',
 'o',
 't',
 ' ',
 's',
 'u',
 'r',
 'e',
 ' ',
 'w',
 'h',
 'o',
 ' ',
 'w',
 'a',
 's',
 ' ',
 'm',
 'o',
 'r',
 'e',
 ' ',
 'l',
 'o',
 's',
 't',
 ' ',
 '-',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'f',
 'l',
 'a',
 't',
 ' ',
 'c',
 'h',
 'a',
 'r',
 'a',
 'c',
 't',
 'e',
 'r',
 's',
 ' ',
 'o',
 'r',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'a',
 'u',
 'd',
 'i',
 'e',
 'n',
 'c',
 'e',
 ',',
 ' ',
 'n',
 'e',
 'a',
 'r',
 'l',
 'y',
 ' ',
 'h',
 'a',
 'l',
 'f',
 ' ',
 'o',
 'f',
 ' ',
 'w',
 'h',
 'o',
 'm',
 ' ',
 'w',
 'a',
 'l',
 'k',
 'e',
 'd',
 ' ',
 'o',
 'u',
 't',
 '.',
 ' ',
 ' ']